In [ ]:
## Pipeline de Ingesta de Datos Financieros desde Yahoo Finance

In [1]:
!pip install yfinance sqlalchemy psycopg2-binary pandas

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 1.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 1.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.9/182.9 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=60f408a0fec6adf8cc90c16823616ac123124d9e805c480edb7745fa015a5c12
  Stored in directory: /home/jovyan/.cache/pip/wheels/42/d6/84/bf57a755f4569494cd00de4bb46ef064874823f4d19c82e960
  Created wheel for p

In [2]:
import os
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine, text
from datetime import datetime
import time

In [6]:
PG_USER = "megranda"
PG_PASSWORD = "TradingDatabase"
PG_HOST = "warehouse"
PG_PORT = "5432"
PG_DB = "trading_db"

engine = create_engine(f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}")

In [ ]:
TICKERS = os.getenv('TICKERS')  
START_DATE = os.getenv('START_DATE') 
END_DATE = os.getenv('END_DATE')

df_all = []

max_retries = 3

for TICKER in TICKERS.split(','):
    # Reintentos para descarga de datos
    for attempt in range(max_retries):
        try:
            print(f"Descargando datos para {TICKER}...")
            df_raw = yf.download(
                tickers=TICKER, 
                start=START_DATE, 
                end=END_DATE, 
                interval='1d',
                progress=False,
                auto_adjust=False
            )

            # Asegurar que las columnas no sean MultiIndex
            if isinstance(df_raw.columns, pd.MultiIndex):
                df_raw.columns = df_raw.columns.get_level_values(0)

            df_raw.reset_index(inplace=True)
            # Añadir columna ticker
            df_raw['ticker'] = TICKER

            df_all.append(df_raw)
            print(f"Filas descargadas para {TICKER}: {len(df_raw)}")
            break  # si descarga ok, rompe el loop de reintentos
        except Exception as e:
            print(f"Error al descargar {TICKER} (intento {attempt + 1}): {e}")
            time.sleep(5 * (attempt + 1))
            if attempt == max_retries - 1:
                print(f"No se pudo descargar {TICKER} después de {max_retries} intentos.")

# Unir todos los DataFrames
df_all = pd.concat(df_all, ignore_index=True)
print(f"Total filas descargadas: {len(df_all)}")

In [ ]:
#Metadatos
df_all['ingested_at_utc'] = datetime.utcnow()
df_all['run_id'] = datetime.now().strftime('run_%Y%m%d_%H%M')
df_all['source_name'] = 'yahoo_finance'